<a href="https://colab.research.google.com/github/tnc-br/ddf-isoscapes/blob/kriging_ensemble_data/kriging_ensemble_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
MEAN_PREDICTED_NAME = 'ordinary_kriging_linear_d18O_predicted_mean' #@param
VAR_PREDICTED_NAME = 'ordinary_kriging_linear_d18O_predicted_variance' #@param

INPUT_PATH = 'canonical/uc_davis_2023_08_26_isoscapes_test_random_grouped.csv' #@param

MEANS_ISOSCAPE_PATH = 'canonical/uc_davis_d18O_cel_ordinary_linear_random_grouped_means.tiff' #@param
VARS_ISOSCAPE_PATH = 'canonical/uc_davis_d18O_cel_ordinary_linear_random_grouped_vars.tiff' #@param

OUTPUT_PATH = 'kriging_ensemble_experiment/uc_davis_2023_08_26_isoscapes_test_random_grouped_ordinary_kriging_linear.csv' #@param


In [2]:
#@title Imports and modules.
%pip install opencv-python
%pip install matplotlib
%pip install pandas

from osgeo import gdal, gdal_array
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
import matplotlib.animation as animation
from matplotlib import rc
from typing import List
from numpy.random import MT19937, RandomState, SeedSequence
import pandas as pd
from tqdm import tqdm
from io import StringIO
import xgboost as xgb
import os
import math
import glob

rc('animation', html='jshtml')

import sys
!if [ ! -d "/content/ddf_common_stub" ] ; then git clone -b test https://github.com/tnc-br/ddf_common_stub.git; fi
sys.path.append("/content/ddf_common_stub/")
import ddfimport
# ddfimport.ddf_source_control_pane()
ddfimport.ddf_import_common()

executing checkout_branch ...
Branch master already checked out.
Remember to reload your imports with `importlib.reload(module)`.
b''
main branch checked out as readonly. You may now use ddf_common imports


In [3]:
import dataset

In [5]:
import raster

In [33]:
df = pd.read_csv(raster.get_sample_db_path(INPUT_PATH))

In [35]:
long = df['long'].values
lat = df['lat'].values

In [36]:
means_isoscape = raster.load_raster(raster.get_raster_path(MEANS_ISOSCAPE_PATH), use_only_band_index=0)
vars_isoscape = raster.load_raster(raster.get_raster_path(VARS_ISOSCAPE_PATH), use_only_band_index=0)

Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)
Driver: GTiff/GeoTIFF
Size is 235 x 234 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0, 5.333333333999995)
Pixel Size = (0.16666666666808508, -0.16666666666808505)


In [37]:
df[MEAN_PREDICTED_NAME] = df.apply(lambda row:raster.get_data_at_coords(means_isoscape, row['long'],row['lat'],-1), axis=1)
df[VAR_PREDICTED_NAME] = df.apply(lambda row:raster.get_data_at_coords(vars_isoscape, row['long'],row['lat'],-1), axis=1)

In [38]:
df

,Unnamed: 0,lat,long,VPD,RH,PET,DEM,PA,Mean Annual Temperature,Mean Annual Precipitation,Iso_Oxi_Stack_mean_TERZER,predkrig_br_lat_ISORG,isoscape_fullmodel_d18O_prec_REGRESSION,Code,Family,Origin,d18O_cel_mean,d18O_cel_variance,ordinary_kriging_linear_d18O_predicted_mean,ordinary_kriging_linear_d18O_predicted_variance
0,90,-2.496000,-59.121000,0.775,0.78866,98.45000,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.84010,mad74,meliaceae,itacoatiara,27.714,0.461230,25.503040,0.661004
1,91,-2.495000,-59.122000,0.775,0.78866,98.45000,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.84010,mad75,meliaceae,itacoatiara,26.820,0.293950,25.503040,0.661004
2,92,-2.495000,-59.121000,0.775,0.78866,98.45000,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.84010,mad76,meliaceae,itacoatiara,26.686,0.267730,25.503040,0.661004
3,93,-2.496000,-59.118000,0.775,0.78866,98.45000,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.84010,mad77,fabaceae,itacoatiara,26.144,0.555280,25.503040,0.661004
4,94,-2.496000,-59.120000,0.775,0.78866,98.45000,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.84010,mad78,fabaceae,itacoatiara,26.134,0.194530,25.503040,0.661004
5,95,-2.493000,-59.121000,0.775,0.78866,98.45000,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.84010,mad79,fabaceae,itacoatiara,25.576,1.794680,25.503040,0.661004
6,96,-3.907183,-66.055146,0.640,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.86485,mad8,burseraceae,jurua,25.196,0.410630,24.951214,0.696812
7,98,-2.497000,-59.121000,0.775,0.78866,98.45000,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.84010,mad81,vochysiaceae,itacoatiara,25.330,0.286467,25.503040,0.661004
8,99,-2.483000,-59.124000,0.775,0.78866,98.45000,96,1001.48932,26.79167,2253,-3.69263,-4.18364,-3.84010,mad82,vochysiaceae,itacoatiara,25.882,0.233670,25.503040,0.661004
9,100,-3.398397,-54.973982,0.645,0.81744,97.93333,139,996.36792,26.00000,1840,-3.30055,-4.49297,-3.42629,mad83,sapotaceae,flona_tapajos,24.792,1.982470,26.124869,0.788921


In [39]:
df.to_csv(raster.get_sample_db_path(OUTPUT_PATH))